In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ''
from openai import OpenAI
client = OpenAI()

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "Cuantos equipos participaron en el mundial de futbol 2018?"},
  ],
    seed=1,
)

In [ ]:
response

ChatCompletion(id='chatcmpl-ANQlCeqzsP4zI8huUCXKZKbeLXM3M', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='32 equipos participaron en el Mundial de Fútbol 2018, celebrado en Rusia.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1730147774, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=23, total_tokens=46, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))

** Llamadas a funciones**     

Las llamadas a funciones de GPT permiten conectar los modelos con herramientas y sistemas externos. Esto es útil para numerosos casos de uso.

**Ejercicio opcional:** Siguiendo el ejemplo brindado, crear tu propias funciones para potenciar al asistente.

**Ejemplo:** Escribir funciones para:
- Permitirle al asistente obtener datos en tiempo real de las condiciones climatológicas en una ubicación determinada. El Asistente deberá poder responder acertadamente a consultas como "¿Cuál es actualmente la temperatura en México DF? Detallar otras condiciones climatológicas de ser posible"
- Permitirle al asistente buscar datos en tiempo real de cotizaciones de criptomonedas.



In [ ]:
!pip install openai

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ''
from openai import OpenAI
client = OpenAI()

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "Cuantos equipos participaron en el mundial de futbol 2018?"},
  ],
    seed=1,
)

In [ ]:
response

ChatCompletion(id='chatcmpl-ANQmenvKtoTuwyywypPC3CaFNHRP4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='32 equipos participaron en el Mundial de Fútbol 2018.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1730147864, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=23, total_tokens=40, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))

In [ ]:
import requests
import json

def get_weather(city):
    api_key = '7663c30fc4c275ea5f373e7052d8a3d3'
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric&lang=es'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'description': data['weather'][0]['main'],
            'humidity': data['main']['humidity'],
            'pressure': data['main']['pressure']
        }
        return weather
    else:
        return {'error': 'Weather data could not be found for the specified city.'}


In [ ]:
clima = get_weather('Mexico')

In [ ]:
clima

{'city': 'Mexico',
 'temperature': 24.84,
 'description': 'Clouds',
 'humidity': 95,
 'pressure': 1006}

In [ ]:
!pip install -U pycoingecko

In [ ]:
from pycoingecko import CoinGeckoAPI

def get_cryptocurrency_price(coin):
    cg = CoinGeckoAPI()
    data = cg.get_price(ids=coin.lower(), vs_currencies='usd')
    if data:
        price = data.get(coin.lower(), {}).get('usd')
        if price:
            return {'coin': coin.capitalize(), 'price_usd': price}
    return {'error': 'Cound not find price for the specified cryptocurrency.'}


In [ ]:
get_cryptocurrency_price('bitcoin')

{'coin': 'Bitcoin', 'price_usd': 69884}

In [ ]:
def ask_gpt_with_tools(user_message):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Gets the current weather of a specific city.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "Name of the city, for example, 'Mexico City'."
                        },
                    },
                    "required": ["city"],
                    "additionalProperties": False,
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_crypto_price",
                "description": "Gets the current price in USD of a specific cryptocurrency.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "currency": {
                            "type": "string",
                            "description": "Name of the cryptocurrency, for example, 'bitcoin', 'ethereum'."
                        }
                    },
                    "required": ["currency"],
                    "additionalProperties": False,
                },
            }
        }
    ]

    messages = [
        {"role": "system", "content": "You are a helpful assistant. Use the supplied tools to assist the user."},
        {"role": "user", "content": user_message}
    ]


    response = OpenAI.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        tools=tools
    )

    mensaje = response["choices"][0]["message"]

    if mensaje.get("function_call"):
        nombre_funcion = mensaje["function_call"]["name"]
        argumentos_funcion = json.loads(mensaje["function_call"]["arguments"])

        if nombre_funcion == "get_weather":
            resultado_funcion = get_weather(argumentos_funcion.get("ciudad"))
        elif nombre_funcion == "get_cryptocurrency_price":
            resultado_funcion = get_cryptocurrency_price(argumentos_funcion.get("moneda"))
        else:
            resultado_funcion = {'error': 'Función no reconocida.'}

        # Añadimos el resultado de la función a la conversación
        mensajes_adicionales = [
            {"role": "user", "content": mensaje_usuario},
            mensaje,
            {
                "role": "function",
                "name": nombre_funcion,
                "content": json.dumps(resultado_funcion, ensure_ascii=False),
            },
        ]

        respuesta_final = OpenAI.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages=mensajes_adicionales,
        )

        return respuesta_final["choices"][0]["message"]["content"]
    else:
        return mensaje["content"]

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Gets the current weather of a specific city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "Name of the city, for example, 'Mexico City'."
                    },
                },
                "required": ["city"],
                "additionalProperties": False,
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_crypto_price",
            "description": "Gets the current price in USD of a specific cryptocurrency.",
            "parameters": {
                "type": "object",
                "properties": {
                    "currency": {
                        "type": "string",
                        "description": "Name of the cryptocurrency, for example, 'bitcoin', 'ethereum'."
                    }
                },
                "required": ["currency"],
                "additionalProperties": False,
            },
        }
    }
]

**Ejercicio:** Programar un chat usando la API de OpenAI. El asistente debe "guardar en memoria" las interacciones previas con el usuario.
Ejemplo:

User: "Me llamo Juan".

Assistant: "Hola Juan!".

User: "Cuál es mi nombre?"

Assistant: "Tu nombre es Juan".


In [2]:
!pip install OpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [22]:
from openai import OpenAI
import os
os.environ["OPENAI_API_KEY"] = ''

client = OpenAI()

messages = []

print("Welcome to the OpenAI Chat! Type 'exit' to end the chat.\n")

while True:
    try:
        user_input = input("User: ")
        if user_input.lower().strip() == "exit":
            print("Chat ended.")
            break

        messages.append({"role": "user", "content": user_input})

        response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=messages
        )

        assistant_message = response.choices[0].message.content.strip()

        print(f"Assistant: {assistant_message}\n")

        messages.append({"role": "assistant", "content": assistant_message})

    except Exception as e:
        print(f"An error ocurred: {e}")
        break


Welcome to the OpenAI Chat! Type 'exit' to end the chat.

User: Hi! My name is Jorge
Assistant: Hello Jorge! How can I assist you today?

User: I want to know a bit about argentinian mate. What is it?
Assistant: Argentinian mate is a traditional South American drink made by infusing dried leaves of the yerba mate plant in hot water. Yerba mate is a species of holly native to South America, particularly Argentina, Uruguay, Paraguay, and southern Brazil. The drink is commonly referred to as just "mate" in Argentina and is typically consumed through a special straw called a bombilla from a hollowed-out gourd known as a mate cup.

Mate is known for its strong, bitter flavor and high caffeine content, making it a popular choice for boosting energy and enhancing mental alertness. It is often enjoyed socially, with friends and family sharing a mate gourd and passing it around in a circle. Drinking mate is considered a deeply ingrained cultural practice in Argentina, and it is often associated

In [26]:
!pip install -U pycoingecko

In [47]:
from pycoingecko import CoinGeckoAPI

def get_crypto_price(currency):
    cg = CoinGeckoAPI()
    data = cg.get_price(ids=currency.lower(), vs_currencies='usd')
    if data:
        price = data.get(currency.lower(), {}).get('usd')
        if price:
            return {'currency': currency.capitalize(), 'price_usd': price}
    return {'error': 'Cound not find price for the specified cryptocurrency.'}


In [28]:
def get_weather(city):
    api_key = ''
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric&lang=es'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'description': data['weather'][0]['main'],
            'humidity': data['main']['humidity'],
            'pressure': data['main']['pressure']
        }
        return weather
    else:
        return {'error': 'Weather data could not be found for the specified city.'}

In [52]:
import json
import requests

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Gets the current weather of a specific city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "Name of the city, for example, 'Mexico City'."
                    },
                },
                "required": ["city"],
                "additionalProperties": False,
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_crypto_price",
            "description": "Gets the current price in USD of a specific cryptocurrency.",
            "parameters": {
                "type": "object",
                "properties": {
                    "currency": {
                        "type": "string",
                        "description": "Name of the cryptocurrency, for example, 'bitcoin', 'ethereum'."
                    }
                },
                "required": ["currency"],
                "additionalProperties": False,
            },
        }
    }
]


messages = []

print("Welcome to the OpenAI Chat with Function Calling! Type 'exit' to end the chat.\n")

while True:
    try:
        user_input = input("User: ")

        if user_input.lower() == 'exit':
            print("Chat ended.")
            break

        messages.append({"role": "user", "content": user_input})

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # Use a model that supports function calling
            messages=messages,
            tools=tools,
        )

        # Extract the assistant's message
        assistant_message = response.choices[0].message

        print(assistant_message)

        # Check if the assistant wants to call a function
        if assistant_message.tool_calls:
            tool_call = assistant_message.tool_calls[0]
            function_args = json.loads(tool_call.function.arguments)
            function_name = tool_call.function.name

            print(f"\nAssistant is calling function: {function_name} with arguments {function_args}\n")

            # Call the appropriate function based on the function name
            if function_name == "get_weather":
                function_response = get_weather(**function_args)
            elif function_name == "get_crypto_price":
                function_response = get_crypto_price(**function_args)
            else:
                function_response = {"error": "Function not recognized."}

            # Convert the function response to a JSON-formatted string
            function_response_json = json.dumps(function_response)

            # Append the assistant's function call and the function's response to the messages
            messages.append(assistant_message)  # Assistant's function call
            function_call_result_message = {
                "role": "tool",
                "content": function_response_json,
                "tool_call_id": tool_call.id
            }
            messages.append(function_call_result_message)

            # Get the assistant's final response using the function's output
            second_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages
            )

            # Extract and print the assistant's final answer
            final_answer = second_response.choices[0].message.content
            print(f"Assistant: {final_answer}\n")

            # Append the final answer to the conversation
            messages.append({"role": "assistant", "content": final_answer})
        else:
            # If no function call, just print the assistant's response
            assistant_content = assistant_message.content.strip()
            print(f"Assistant: {assistant_content}\n")
            messages.append(assistant_message)

    except Exception as e:
        print(f"An error occurred: {e}")
        break


Welcome to the OpenAI Chat with Function Calling! Type 'exit' to end the chat.

User: i want to know mexico weather
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bLjXIuc2WAxiuLqmhpzsyN9M', function=Function(arguments='{"city":"Mexico City"}', name='get_weather'), type='function')])

Assistant is calling function: get_weather with arguments {'city': 'Mexico City'}

Assistant: The current weather in Mexico City is 22.75°C with cloudy skies. The humidity is at 41% and the atmospheric pressure is 1016 hPa.

User: and the weather in rosario?
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7lztDK9SmcLb8hmJTkPHBI4F', function=Function(arguments='{"city":"Rosario"}', name='get_weather'), type='function')])

Assistant is calling function: get_weather with arguments {'city': 'Rosario'}

Assi